In [1]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import zmq
import json
import tensorflow as tf

model_path = "test_15M.h5"
model = tf.keras.models.load_model(model_path)

file_path = "scaler_params.npy"
scaler = MinMaxScaler()
scaler.min_, scaler.scale_ = np.load(file_path)

context = zmq.Context()
socket = context.socket(zmq.REP)
socket.bind("tcp://*:8001")

def predict_price(close_prices):
    close_normalized = scaler.transform(np.array(close_prices).reshape(-1, 1))

    prediction_normalized = model.predict(np.array([close_normalized]).reshape(1, -1, 1))

    predicted_price = scaler.inverse_transform(prediction_normalized)[0][0]

    return predicted_price

while True:
    message = socket.recv()
    json_data = json.loads(message)

    close_prices = [
        json_data['closePrice1'],
        json_data['closePrice2'],
        json_data['closePrice3'],
        json_data['closePrice4'],
        json_data['closePrice5']
    ]

    predicted_price = predict_price(close_prices)
    print("Received request:", close_prices)

    socket.send_string(str(predicted_price))



1/1 [==============================] - 1s 1s/step
Received request: [2181.89, 2181.46, 2181.29, 2181.94, 2183.07]
1/1 [==============================] - 0s 192ms/step
Received request: [70988.99, 70697.73, 70591.37, 70476.84, 69268.84]
1/1 [==============================] - 0s 47ms/step
Received request: [2181.89, 2181.46, 2181.29, 2181.94, 2183.07]
1/1 [==============================] - 0s 31ms/step
Received request: [72210.43, 72066.29, 71998.09, 72248.12, 72487.79]
1/1 [==============================] - 0s 34ms/step
Received request: [2181.89, 2181.46, 2181.29, 2181.94, 2183.07]
1/1 [==============================] - 0s 30ms/step
Received request: [2181.89, 2181.46, 2181.29, 2181.94, 2183.07]
1/1 [==============================] - 0s 30ms/step
Received request: [2181.89, 2181.46, 2181.29, 2181.94, 2183.07]
1/1 [==============================] - 0s 31ms/step
Received request: [2181.89, 2181.46, 2181.29, 2181.94, 2183.07]
1/1 [==============================] - 0s 80ms/step
Received 